In [18]:
import pandas as pd
import arrow
from dateutil import tz
import numpy as np
import plotly.express as px


In [2]:
system_power = 5 #kW
system_capacity = 13.5 #kWh
cycling_charge = 0.07 #$/kWh

years = [2016, 2017, 2018, 2019, 2020]
states = ['SA', 'VIC', 'NSW', 'QLD']
results = {
    'SA': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'VIC': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'NSW': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'QLD': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    }

In [3]:
# Step 1, general savings for spot exposed customer, taken from amberelectric.com.au front page

amber_dmo_adjustment = {
    'SA': 1850-1600,
    'VIC': 1325-1100,
    'NSW': 1475-1225,
    'QLD': 1575-1400
    }
for k,v in amber_dmo_adjustment.items():
    for year in years:
        results[k][year]['dmo_adjustment'] = v


,Unnamed: 0,SETTLEMENTDATE,REGIONID,PERIODID,RRP,LASTCHANGED,RAISE6SECRRP,RAISE60SECRRP,RAISE5MINRRP,LOWER6SECRRP,LOWER60SECRRP,LOWER5MINRRP,PRICE_STATUS
SETTLEMENTDATE,,,,,,,,,,,,,
2020-01-01 00:30:00,0,2020-01-01 00:30:00,SA1,1,68.33,2020/01/01 00:25:03,1.75,2.47,0.73,0.05,0.38,0.44,FIRM
2020-01-01 01:00:00,1,2020-01-01 01:00:00,SA1,2,68.33,2020/01/01 00:55:03,1.85,2.80,0.75,0.46,0.45,0.71,FIRM
2020-01-01 01:30:00,2,2020-01-01 01:30:00,SA1,3,68.76,2020/01/01 01:25:03,1.94,2.80,0.73,0.47,0.45,0.98,FIRM
2020-01-01 02:00:00,3,2020-01-01 02:00:00,SA1,4,68.42,2020/01/01 01:55:03,1.82,2.80,0.70,0.53,0.45,1.02,FIRM
2020-01-01 02:30:00,4,2020-01-01 02:30:00,SA1,5,68.25,2020/01/01 02:25:03,1.74,2.30,0.70,0.54,0.45,1.02,FIRM


In [4]:
for state in states:
    df = pd.read_csv("files/{}12016-2020.csv".format(state),parse_dates=['SETTLEMENTDATE'])
    for year in years:
        results[state][year]['wholesale_peak'] = 0
    df.index = df['SETTLEMENTDATE']
    DFList = [group[1] for group in df.groupby(df.index.date)]
    for day in DFList:
        day = day[day['RRP'] > 1000]
        if len(day) > 0:
            price = day['RRP'].mean()/1000 #  average peak price converted to $/kWh 
            duration = min(2, len(day)/2)/2 # hours of peak pricing generated against
            profit = system_power * price * duration
            results[state][day.index.year[0]]['wholesale_peak'] += profit
        #print(int(day.index.year[0]))
#DFList.head()
results


{'SA': {2016: {'dmo_adjustment': 250, 'wholesale_peak': 226.65393107769418},
  2017: {'dmo_adjustment': 250, 'wholesale_peak': 139.3492589285714},
  2018: {'dmo_adjustment': 250, 'wholesale_peak': 155.073708234127},
  2019: {'dmo_adjustment': 250, 'wholesale_peak': 177.43406726190474},
  2020: {'dmo_adjustment': 250, 'wholesale_peak': 77.514905}},
 'VIC': {2016: {'dmo_adjustment': 225, 'wholesale_peak': 30.791512500000003},
  2017: {'dmo_adjustment': 225, 'wholesale_peak': 5.042350000000001},
  2018: {'dmo_adjustment': 225, 'wholesale_peak': 90.48939083333333},
  2019: {'dmo_adjustment': 225, 'wholesale_peak': 153.72292083333335},
  2020: {'dmo_adjustment': 225, 'wholesale_peak': 94.51080416666666}},
 'NSW': {2016: {'dmo_adjustment': 250, 'wholesale_peak': 31.3945},
  2017: {'dmo_adjustment': 250, 'wholesale_peak': 94.2167},
  2018: {'dmo_adjustment': 250, 'wholesale_peak': 15.279587500000002},
  2019: {'dmo_adjustment': 250, 'wholesale_peak': 15.280225},
  2020: {'dmo_adjustment': 250

In [5]:
for state in states:
    df = pd.read_csv("files/{}12016-2020.csv".format(state),parse_dates=['SETTLEMENTDATE'])
    for year in years:
        results[state][year]['precharging'] = {'count': 0, 'value': 0}
    df.index = df['SETTLEMENTDATE']
    DFList = [group[1] for group in df.groupby(df.index.date)]
    for day in DFList:
        daylight_hours = day.between_time('8:00', '16:00')
        daylight_hours_cloud = daylight_hours['CLOUD'].mean()
        if daylight_hours_cloud > 0.7:
            day = day[day['RRP'] < 1000] # don't include offseting super peak prices, this is done in wholesale_peak
            precharging_price = day['RRP'].nlargest(2).mean()/1000 - day['RRP'].nsmallest(2).mean()/1000
            if precharging_price > cycling_charge:
                results[state][day.index.year[0]]['precharging']['value'] += precharging_price * system_capacity / 2
                results[state][day.index.year[0]]['precharging']['count'] += 1

        #print(int(day.index.year[0]))
#DFList.head()
#results

{'SA': {2016: {'dmo_adjustment': 250,
   'wholesale_peak': 226.65393107769418,
   'precharging': {'count': 111, 'value': 134.26715250000004}},
  2017: {'dmo_adjustment': 250,
   'wholesale_peak': 139.3492589285714,
   'precharging': {'count': 86, 'value': 94.1220675}},
  2018: {'dmo_adjustment': 250,
   'wholesale_peak': 155.073708234127,
   'precharging': {'count': 100, 'value': 115.51234499999995}},
  2019: {'dmo_adjustment': 250,
   'wholesale_peak': 177.43406726190474,
   'precharging': {'count': 103, 'value': 144.44136}},
  2020: {'dmo_adjustment': 250,
   'wholesale_peak': 77.514905,
   'precharging': {'count': 79, 'value': 93.08719125}}},
 'VIC': {2016: {'dmo_adjustment': 225,
   'wholesale_peak': 30.791512500000003,
   'precharging': {'count': 98, 'value': 102.220515}},
  2017: {'dmo_adjustment': 225,
   'wholesale_peak': 5.042350000000001,
   'precharging': {'count': 97, 'value': 87.37621874999996}},
  2018: {'dmo_adjustment': 225,
   'wholesale_peak': 90.48939083333333,
   'p

In [19]:
for state in states:
    sa1 = {'years': [], 'spot_savings': [], 'peak_earnings': [], 'precharging': []}
    for year in years:
        sa1['years'].append(year)
        sa1['spot_savings'].append(results[state][year]['dmo_adjustment'])
        sa1['peak_earnings'].append(results[state][year]['wholesale_peak'])
        sa1['precharging'].append(results[state][year]['precharging']['value'])
    dfsa = pd.DataFrame(data=sa1)
    dfsa.head()
    wide_df = dfsa

    fig = px.bar(wide_df, x="years", y=["spot_savings", "peak_earnings", "precharging"], title="{} Savings".format(state))
    fig.show()
    fig.write("images/{}-savings.png".format(state))

AttributeError: 'Figure' object has no attribute 'write'